In [1]:
import json
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
from mapboxgl.utils import *
import h3
import geopandas as gpd
import plotly.express as px
import folium
import matplotlib.pyplot as plt
from folium import Map, Marker, GeoJson
import branca.colormap as cm
from geojson.feature import *
from shapely.geometry import Polygon
import geojson
from shapely import geometry, ops
from keplergl import KeplerGl

import warnings
warnings.filterwarnings('ignore')


# don't use scientific notation
np.set_printoptions(suppress=True) 
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [2]:
pip install mapboxgl

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install h3

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install geopandas

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install keplergl

Note: you may need to restart the kernel to use updated packages.


In [6]:
f = open('vsl_arrivals_020920.json')
data = json.load(f)

In [7]:
df_arrival = pd.DataFrame.from_dict(json_normalize(data), orient='columns')
df_arrival.head()

,location,grid,purpose,agent,reportedArrivalTime,vesselParticulars.vesselName,vesselParticulars.callSign,vesselParticulars.imoNumber,vesselParticulars.flag
0,EASTERN PETROL ANCH A,5415D,"N,N,Y,N,N,N,N,N,N,",FOCAL SHIPPING SERVICES PTE LTD,2020-09-02 00:00:00,STI EXPEDITE,V7IS4,9735593,MH
1,WESTERN PETRO ANCH A,4714B,"N,N,N,Y,Y,N,N,N,N,",JOINT WIN SHIPPING AGENCY & SHIPMAN,2020-09-02 00:05:00,SOUTHERNPEC 9,9V9208,9613977,SG
2,PASIR PANJANG BERTH 39,,"Y,N,N,N,Y,N,N,N,N,",COSCO SHIPPING LINES SINGAPORE PTE,2020-09-02 00:15:00,COSCO ADEN,VRKF9,9484003,HK
3,EASTERN BUNKERG ANCH B,5917D,"N,N,Y,N,N,N,N,N,N,",CMA CGM & ANL (SINGAPORE) PTE LTD,2020-09-02 00:20:00,BALTIC SOUTH,V7A2516,9463281,MH
4,PASIR PANJANG BERTH 35,,"Y,N,Y,N,Y,N,N,N,N,",COSCO SHIPPING LINES SINGAPORE PTE,2020-09-02 00:20:00,BANGKOK,3FAJ3,9197349,PA


In [8]:
df1=df_arrival.purpose.str.split(',', expand=(True))
df_arrival=df_arrival.join(df1, on=None, how='left')

In [9]:
df_arrival.columns=['location','grid','purpose','agent','reportedArrivalTime','vesselName',
           'callsign','imoNumber','flag',
           'L/UCargo','L/UPassenger','Bunkering','ShipSupply','CrewChange','ShipyardRepair','OffshoreSupport',
   'NotUsed','OtherAfloatActivities','']

df_arrival.head()

,location,grid,purpose,agent,reportedArrivalTime,vesselName,callsign,imoNumber,flag,L/UCargo,L/UPassenger,Bunkering,ShipSupply,CrewChange,ShipyardRepair,OffshoreSupport,NotUsed,OtherAfloatActivities,
0,EASTERN PETROL ANCH A,5415D,"N,N,Y,N,N,N,N,N,N,",FOCAL SHIPPING SERVICES PTE LTD,2020-09-02 00:00:00,STI EXPEDITE,V7IS4,9735593,MH,N,N,Y,N,N,N,N,N,N,
1,WESTERN PETRO ANCH A,4714B,"N,N,N,Y,Y,N,N,N,N,",JOINT WIN SHIPPING AGENCY & SHIPMAN,2020-09-02 00:05:00,SOUTHERNPEC 9,9V9208,9613977,SG,N,N,N,Y,Y,N,N,N,N,
2,PASIR PANJANG BERTH 39,,"Y,N,N,N,Y,N,N,N,N,",COSCO SHIPPING LINES SINGAPORE PTE,2020-09-02 00:15:00,COSCO ADEN,VRKF9,9484003,HK,Y,N,N,N,Y,N,N,N,N,
3,EASTERN BUNKERG ANCH B,5917D,"N,N,Y,N,N,N,N,N,N,",CMA CGM & ANL (SINGAPORE) PTE LTD,2020-09-02 00:20:00,BALTIC SOUTH,V7A2516,9463281,MH,N,N,Y,N,N,N,N,N,N,
4,PASIR PANJANG BERTH 35,,"Y,N,Y,N,Y,N,N,N,N,",COSCO SHIPPING LINES SINGAPORE PTE,2020-09-02 00:20:00,BANGKOK,3FAJ3,9197349,PA,Y,N,Y,N,Y,N,N,N,N,


In [10]:
f1 = open(r'vsl_positions_0800.json')
f2 = open(r'vsl_positions_0900.json')
f3 = open(r'vsl_positions_1100.json')
f4 = open(r'vsl_positions_1200.json')
f5 = open(r'vsl_positions_1300.json')
f6 = open(r'vsl_positions_1400.json')
f7 = open(r'vsl_positions_1500.json')

data1 = json.load(f1)
data2 = json.load(f2)
data3 = json.load(f3)
data4 = json.load(f4)
data5 = json.load(f5)
data6 = json.load(f6)
data7 = json.load(f7)

In [11]:
df_positions_1 = pd.DataFrame.from_dict(json_normalize(data1), orient='columns')
df_positions_2 = pd.DataFrame.from_dict(json_normalize(data2), orient='columns')
df_positions_3 = pd.DataFrame.from_dict(json_normalize(data3), orient='columns')
df_positions_4 = pd.DataFrame.from_dict(json_normalize(data4), orient='columns')
df_positions_5 = pd.DataFrame.from_dict(json_normalize(data5), orient='columns')
df_positions_6 = pd.DataFrame.from_dict(json_normalize(data6), orient='columns')
df_positions_7 = pd.DataFrame.from_dict(json_normalize(data7), orient='columns')
df_positions=pd.concat([df_positions_1,df_positions_2,df_positions_3,df_positions_4,df_positions_5, df_positions_6, df_positions_7])
df_positions.head()

,latitude,longitude,latitudeDegrees,longitudeDegrees,speed,course,heading,timeStamp,vesselParticulars.vesselName,vesselParticulars.callSign,...,vesselParticulars.flag,vesselParticulars.vesselLength,vesselParticulars.vesselBreadth,vesselParticulars.vesselDepth,vesselParticulars.vesselType,vesselParticulars.grossTonnage,vesselParticulars.netTonnage,vesselParticulars.deadweight,vesselParticulars.mmsiNumber,vesselParticulars.yearBuilt
0,0.02225,1.81094,1.27489,103.75947,0.00000,0.00000,300.99800,2020-09-02 08:03:31,MATTINA,V7FQ4,...,MH,264,0.00000,0.00000,CS,42484,25512,0,538008047,2007
1,0.02252,1.81389,1.29039,103.92821,0.09719,143.13000,91.99950,2020-09-02 08:03:31,NEVA,V7JA2,...,MH,176,30.00000,0.00000,BC,19728,11391,31824,538008106,2000
2,0.02268,1.81091,1.29944,103.75751,0.00000,0.00000,43.99480,2020-09-02 08:03:31,BRIDGEWATER 130,9V5660,...,SG,70,17.00000,7.50000,TU,2708,812,0,538003493,2008
3,0.02148,1.81341,1.23058,103.90053,1.28308,270.86800,243.99500,2020-09-02 08:03:31,KOTA NABIL,9VFX3,...,SG,180,28.00000,15.90000,CS,20902,9119,0,565795000,2007
4,0.02182,1.80643,1.25034,103.50066,0.08693,153.43500,113.00000,2020-09-02 08:03:31,NEW GLOBAL,V7A2122,...,MH,332,58.00000,0.00000,TA,159016,100899,305704,538008237,2001


In [12]:
df_positions.columns=['latitude', 'longitude', 'latitudeDegrees', 'longitudeDegrees', 'speed',
       'course', 'heading', 'timeStamp', 'vesselName',
       'callSign', 'imoNumber',
       'flag', 'vesselLength',
       'vesselBreadth', 'vesselDepth',
       'vesselType', 'grossTonnage',
       'netTonnage', 'deadweight',
       'mmsiNumber', 'yearBuilt']

In [13]:
gdf=gpd.GeoDataFrame(df_positions, geometry=gpd.points_from_xy(df_positions.longitudeDegrees,df_positions.latitudeDegrees))

In [14]:
vessel_df=pd.merge(gdf, df_arrival, on="vesselName", how="inner")

vessel_df=vessel_df.replace({'Y': 1, 'N': 0})
vessel_df.head()

,latitude,longitude,latitudeDegrees,longitudeDegrees,speed,course,heading,timeStamp,vesselName,callSign,...,L/UCargo,L/UPassenger,Bunkering,ShipSupply,CrewChange,ShipyardRepair,OffshoreSupport,NotUsed,OtherAfloatActivities,
0,0.02252,1.81389,1.29039,103.92821,0.09719,143.13000,91.99950,2020-09-02 08:03:31,NEVA,V7JA2,...,0,0,1,0,0,0,0,0,0,
1,0.02251,1.81389,1.28972,103.92834,0.09719,180.00000,70.99910,2020-09-02 09:18:25,NEVA,V7JA2,...,0,0,1,0,0,0,0,0,0,
2,0.02210,1.81395,1.26600,103.93174,5.36843,140.14100,136.00000,2020-09-02 11:06:21,NEVA,V7JA2,...,0,0,1,0,0,0,0,0,0,
3,0.02059,1.81183,1.18000,103.81032,12.09320,244.17600,239.99600,2020-09-02 12:05:17,NEVA,V7JA2,...,0,0,1,0,0,0,0,0,0,
4,0.02069,1.80909,1.18556,103.65326,11.75380,291.13300,287.99600,2020-09-02 12:57:27,NEVA,V7JA2,...,0,0,1,0,0,0,0,0,0,


In [15]:
vessel_df['status']=np.where(vessel_df['speed']>0,'under way','anchored/moored')

In [16]:
def load_and_prepare_anchorages(filepath):
    """Loads a geojson files of polygon geometries and features,
    swaps the latitude and longitude andstores geojson"""

    gdf_anchorages = gpd.read_file(filepath, driver="GeoJSON")
    
    gdf_anchorages["geom_geojson"] = gdf_anchorages["geometry"].apply(lambda x: geometry.mapping(x))

    gdf_anchorages["geom_swap"] = gdf_anchorages["geometry"].map(lambda polygon: ops.transform(lambda x, y: (y, x), polygon))

    gdf_anchorages["geom_swap_geojson"] = gdf_anchorages["geom_swap"].apply(lambda x: geometry.mapping(x))
    
    return gdf_anchorages

In [17]:
input_file_districts = "singapore_anch_new.geojson"
gdf_anchorages = load_and_prepare_anchorages(filepath = input_file_districts) 
 
#print(gdf_anchorages.shape)
#print("\n--------------------------------------------------------\n")
#list_districts = list(gdf_districts["libelle_du_grand_quartier"].unique())
#list_districts.sort()
#print(columnize(list_districts, displaywidth=100))
#print("\n--------------------------------------------------------\n")

gdf_anchorages[["name", "purpose", "geometry", 
                "geom_swap", "geom_swap_geojson"]]

,name,purpose,geometry,geom_swap,geom_swap_geojson
0,ACBTH,For barges with sand/granite,"POLYGON ((1.348 104.051, 1.340 104.047, 1.336 ...","POLYGON ((104.05056 1.34758, 104.04666 1.33967...","{'type': 'Polygon', 'coordinates': (((104.0505..."
1,ACGP,For general purpose,"POLYGON ((1.333 104.052, 1.315 104.070, 1.315 ...","POLYGON ((104.05250 1.33326, 104.06960 1.31462...","{'type': 'Polygon', 'coordinates': (((104.0525..."
2,AMOW,For visiting warships,"POLYGON ((1.315 104.059, 1.306 104.059, 1.307 ...","POLYGON ((104.05920 1.31471, 104.05920 1.30616...","{'type': 'Polygon', 'coordinates': (((104.0592..."
3,AEBA,"For vessels >= 20,000 GT, other than LPG, LNG,...","POLYGON ((1.306 104.046, 1.289 104.046, 1.297 ...","POLYGON ((104.04650 1.30590, 104.04650 1.28883...","{'type': 'Polygon', 'coordinates': (((104.0465..."
4,AEBB,"For vessels >= 20,000 GT, other than LPG, LNG,...","POLYGON ((1.292 103.982, 1.277 103.982, 1.286 ...","POLYGON ((103.98200 1.29161, 103.98200 1.27710...","{'type': 'Polygon', 'coordinates': (((103.982,..."
5,AEPBC,"For tankers < 10,000 GT loaded with petroleum,...","POLYGON ((1.300 103.983, 1.292 103.983, 1.294 ...","POLYGON ((103.98290 1.30000, 103.98290 1.29161...","{'type': 'Polygon', 'coordinates': (((103.9829..."
6,ASCB,"For harbour tugs, pontoons, barges and other s...","POLYGON ((1.305 103.983, 1.300 103.982, 1.300 ...","POLYGON ((103.98290 1.30495, 103.98200 1.30000...","{'type': 'Polygon', 'coordinates': (((103.9829..."
7,ASCA,"For harbour tugs, pontoons, barges and other s...","POLYGON ((1.305 103.956, 1.300 103.943, 1.300 ...","POLYGON ((103.95555 1.30495, 103.94288 1.30000...","{'type': 'Polygon', 'coordinates': (((103.9555..."
8,AEPBB,"For tankers < 10,000 GT loaded with petroleum,...","POLYGON ((1.300 103.943, 1.296 103.933, 1.292 ...","POLYGON ((103.94280 1.30000, 103.93330 1.29626...","{'type': 'Polygon', 'coordinates': (((103.9428..."
9,AESPA,"For vessels under arrest, damaged vessels, dee...","POLYGON ((1.296 103.933, 1.287 103.908, 1.277 ...","POLYGON ((103.93330 1.29620, 103.90815 1.28670...","{'type': 'Polygon', 'coordinates': (((103.9333..."


In [18]:
def counts_by_hexagon(df, resolution):
    """
    Use h3.geo_to_h3 to index each data point into the spatial index of the specified resolution.
    Use h3.h3_to_geo_boundary to obtain the geometries of these hexagons
    
    Ex counts_by_hexagon(data, 9)
    """
    df = df[["latitudeDegrees","longitudeDegrees"]]
    
    df["hex_id"] = df.apply(lambda row: h3.geo_to_h3(row["latitudeDegrees"], row["longitudeDegrees"], resolution), axis = 1)
    
    df_aggreg = df.groupby(by = "hex_id").size().reset_index()
    df_aggreg.columns = ["hex_id", "value"]
    
    df_aggreg['geometry'] = df_aggreg.hex_id.apply(lambda x: {'type' : 'Polygon','coordinates': [h3.h3_to_geo_boundary(h=x,geo_json=True)]})
    return df_aggreg


def hexagons_dataframe_to_geojson(df_hex, file_output = None):
    """
    Produce the GeoJSON for a dataframe that has a geometry column in geojson 
    format already, along with the columns hex_id and value
    
    Ex counts_by_hexagon(data)
    """    
    list_features = []
    
    for i,row in df_hex.iterrows():
        feature = Feature(geometry = row["geometry"] , id=row["hex_id"], properties = {"value" : row["value"]})
        list_features.append(feature)
        
    feat_collection = FeatureCollection(list_features)
    
    geojson_result = json.dumps(feat_collection)
    
    #optionally write to file
    # if file_output is not None:
    #     with open(file_output,"w") as f:
    #         json.dump(feat_collection,f)
    
    return geojson_result


def choropleth_map(df_aggreg, df_boundaries_zones, border_color = 'black', fill_opacity = 0.7, initial_map = None, with_legend = False,
                   kind = "linear"):
    
    """
    Creates choropleth maps given the aggregated data.
    """    
    #colormap
    min_value = df_aggreg["value"].min()
    max_value = df_aggreg["value"].max()
    m = round ((min_value + max_value ) / 2 , 0)
    
    #take resolution from the first row
    res = h3.h3_get_resolution(df_aggreg.loc[0,'hex_id'])
    
    if initial_map is None:
        initial_map = Map(location= [1.088375,	103.071017], zoom_start=12, tiles="cartodbdark_matter")
        

    #the colormap 
    #color names accepted https://github.com/python-visualization/branca/blob/master/branca/_cnames.json
    if kind == "linear":
        custom_cm = cm.LinearColormap(['green','yellow','red'], vmin=min_value, vmax=max_value)
    elif kind == "outlier":
        #for outliers, values would be -11,0,1
        custom_cm = cm.LinearColormap(['blue','white','red'], vmin=min_value, vmax=max_value)
    elif kind == "filled_nulls":
        custom_cm = cm.LinearColormap(['sienna','green','yellow','red'], 
                                      index=[0,min_value,m,max_value],vmin=min_value,vmax=max_value)
   

    #create geojson data from dataframe
    geojson_data = hexagons_dataframe_to_geojson(df_hex = df_aggreg)
    
    #plot on map
    name_layer = "Choropleth " + str(res)
    if kind != "linear":
        name_layer = name_layer + kind
        
    GeoJson(
        geojson_data,
        style_function=lambda feature: {
            'fillColor': custom_cm(feature['properties']['value']),
            'color': border_color,
            'weight': 1,
            'fillOpacity': fill_opacity 
        }, 
        name = name_layer
    ).add_to(initial_map)
    #add legend (not recommended if multiple layers)
    if with_legend == True:
        custom_cm.add_to(initial_map)
    
    feature_group = folium.FeatureGroup(name='Boundaries')

    for i, row in df_boundaries_zones.iterrows():
        feature_sel = Feature(geometry = row["geom_swap"], id=str(i))
        feat_collection_sel = FeatureCollection([feature_sel])
        geojson_subzone = json.dumps(feat_collection_sel)

        GeoJson(
                geojson_subzone,
                style_function=lambda feature: {
                    'fillColor': None,
                    'color': 'blue',
                    'weight': 5,
                    'fillOpacity': 0
                }
            ).add_to(feature_group)

    feature_group.add_to(initial_map)
    
    
    return initial_map
    



In [19]:
df_aggreg = counts_by_hexagon(df = vessel_df, resolution = 9)
df_aggreg.sort_values(by = "value", ascending = False, inplace = True)
# Creates a map using Folium
hexmap = choropleth_map(df_aggreg = df_aggreg, df_boundaries_zones=gdf_anchorages, with_legend = True)

In [20]:
df_aggreg_9 = counts_by_hexagon(df = vessel_df, resolution = 14)
df_aggreg_8 = counts_by_hexagon(df = vessel_df, resolution = 10)
df_aggreg_7 = counts_by_hexagon(df = vessel_df, resolution = 9)
df_aggreg_6 = counts_by_hexagon(df = vessel_df, resolution =8)


hexmap9 = choropleth_map(df_aggreg = df_aggreg_9,df_boundaries_zones=gdf_anchorages,with_legend = False)
hexmap8 = choropleth_map(df_aggreg = df_aggreg_8,df_boundaries_zones=gdf_anchorages,initial_map = hexmap9, with_legend = False)
hexmap7 = choropleth_map(df_aggreg = df_aggreg_7,df_boundaries_zones=gdf_anchorages, initial_map = hexmap8 , with_legend = False)
hexmap6 = choropleth_map(df_aggreg = df_aggreg_6,df_boundaries_zones=gdf_anchorages, initial_map = hexmap7, with_legend = False)

folium.map.LayerControl('bottomright', collapsed=False).add_to(hexmap6)

hexmap9

In [21]:
vessel_df["hex_id"] = vessel_df.apply(lambda row: h3.geo_to_h3(row["latitudeDegrees"], row["longitudeDegrees"], 9), axis = 1)

In [22]:
map_1 = KeplerGl(height=630)
map_1.add_data(data=vessel_df, name='data_1')
map_1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data_1': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, …